# Setup

In [1]:
# !pip install --quiet --upgrade google-cloud-logging google_cloud_firestore google_cloud_aiplatform langchain langchain-google-vertexai langchain_community langchain_experimental pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.4/159.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [7]:
!pip uninstall -y langchain langchain-core langchain-community langchain-experimental \
  langchain-google-vertexai google-cloud-aiplatform google-cloud-firestore protobuf || true

Found existing installation: langchain 1.0.2
Uninstalling langchain-1.0.2:
  Successfully uninstalled langchain-1.0.2
Found existing installation: langchain-core 1.0.0
Uninstalling langchain-core-1.0.0:
  Successfully uninstalled langchain-core-1.0.0
Found existing installation: langchain-community 0.4
Uninstalling langchain-community-0.4:
  Successfully uninstalled langchain-community-0.4
Found existing installation: langchain-experimental 0.0.42
Uninstalling langchain-experimental-0.0.42:
  Successfully uninstalled langchain-experimental-0.0.42
Found existing installation: langchain-google-vertexai 3.0.0
Uninstalling langchain-google-vertexai-3.0.0:
  Successfully uninstalled langchain-google-vertexai-3.0.0
Found existing installation: google-cloud-aiplatform 1.122.0
Uninstalling google-cloud-aiplatform-1.122.0:
  Successfully uninstalled google-cloud-aiplatform-1.122.0
Found existing installation: google-cloud-firestore 2.21.0
Uninstalling google-cloud-firestore-2.21.0:
  Successful

In [9]:
# 1) Clean, correct, and pin versions known to work together
!pip install -U \
  "protobuf<6.30,>=4.25.5" \
  google-cloud-logging \
  google-cloud-firestore \
  google-cloud-aiplatform \
  langchain==1.* \
  langchain-google-vertexai \
  langchain-community \
  langchain-experimental \
  langchain-google-firestore \
  pymupdf

  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metadata (592 bytes)
  Using cached google_cloud_firestore-2.21.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached google_cloud_aiplatform-1.122.0-py2.py3-none-any.whl.metadata (44 kB)
  Using cached langchain-1.0.2-py3-none-any.whl.metadata (4.7 kB)
  Using cached langchain_google_vertexai-3.0.0-py3-none-any.whl.metadata (5.2 kB)
  Using cached langchain_community-0.4-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_experimental-0.3.4-py3-none-any.whl.metadata (1.7 kB)
  Using cached langchain_google_firestore-0.5.0-py3-none-any.whl.metadata (19 kB)
  Using cached langchain_core-1.0.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached langchain_community-0.3.31-py3-none-any.whl.metadata (3.0 kB)
INFO: pip is looking at multiple versions of langchain-experimental to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_experimental-0.3.3-py3-none-any.w

In [27]:
import pickle
from IPython.display import display, Markdown
import logging
import google.cloud.logging
import vertexai

from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel

# LangChain integrations
from langchain_google_vertexai import VertexAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_experimental.text_splitter import SemanticChunker

# Firestore native client & vector search
from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

ModuleNotFoundError: No module named 'langchain_experimental.text_splitter'

In [26]:
# import google.cloud.logging, google.cloud.firestore
# import langchain, langchain_community, langchain_experimental
# import langchain_google_vertexai, vertexai
# from google.cloud.firestore_v1.base_vector_query import DistanceMeasure

# print("OK ✅",
#       google.cloud.firestore.__version__,
#       langchain.__version__,
#       langchain_google_vertexai.__version__)

# Setup with old langchain

In [12]:
# (Optional) Clean out conflicting wheels
!pip uninstall -y \
  langchain langchain-core langchain-community langchain-experimental \
  langchain-google-vertexai langchain-google-firestore \
  google-cloud-firestore google-cloud-aiplatform protobuf || true

In [14]:
# Pre‑v1 LangChain stack with Vertex AI + Firestore vector search
!pip install --upgrade --no-cache-dir \
  "protobuf<6.30,>=4.25.5" \
  google-cloud-logging \
  google-cloud-firestore==2.21.0 \
  google-cloud-aiplatform \
  "langchain==0.2.*" \
  "langchain-community==0.2.*" \
  "langchain-experimental>=0.0.61,<0.0.70" \
  "langchain-google-vertexai==2.*" \
  langchain-google-firestore \
  pymupdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 210.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-google-vertexai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-vertexai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible

In [ ]:
# Next, initialize Vertex AI with your project-id qwiklabs-gcp-04-937bce1e7c69 and a location of us-central1.


# then
# Populate a variable named embedding_model with an instance of the langchain_google_vertexai class VertexAIEmbeddings. Pass it a parameter model_name set to the text embedding model version of text-embedding-005. You will use this LangChain class for your embedding model so that you can use a LangChain semantic chunker to chunk your dataset.
# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/
# https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models#models

# pls be careful with langchain due to an update in langchain to v1.0 in september 2025 which can break old code

In [31]:
%%bash
cat > constraints-preV1.txt <<'EOF'
# Avoid protobuf 6.30+ API removals
protobuf<6.30,>=4.25.5

# Firestore client with Vector / DistanceMeasure / find_nearest
google-cloud-firestore==2.21.0

# Vertex AI SDK
google-cloud-aiplatform

# === Pre‑v1 LangChain family ===
# last 0.2.x on PyPI
langchain==0.2.17
# community from the same era; requires langchain >=0.2.17,<0.3.0
langchain-community==0.2.19
# experimental from the same era (SemanticChunker ok)
langchain-experimental==0.0.62

# Vertex AI integration matching 0.2.x (NOT 3.x)
langchain-google-vertexai==2.0.18

# Firestore VectorStore integration
langchain-google-firestore

# PDF parser used by PyMuPDFLoader
pymupdf
EOF

In [32]:
!pip install --upgrade --no-cache-dir -r constraints-preV1.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-google-vertexai to determine which version is compatible with other requirements. This could take a while.
ERROR: Cannot install -r constraints-preV1.txt (line 12), -r constraints-preV1.txt (line 14), -r constraints-preV1.txt (line 16) and -r constraints-preV1.txt (line 19) because these package versions have conflicting dependencies.

The conflict is caused by:
    langchain 0.2.17 depends on langchain-core<0.3.0 and >=0.2.43
    langchain-community 0.2.19 depends on langchain-core<0.3.0 and >=0.2.43
    langchain-experimental 0.0.62 depends on langchain-core<0.3.0 and >=0.2.10
    langchain-google-vertexai 2.0.18 depends on langchain-core<0.4 and >=0.3.49

To fix this you could try to:
1. loosen the range of package versions you've specified
2. remove package versions to allow pip to attempt to solve the dependency conflict

ERROR: ResolutionImpossibl

In [24]:
pip list

Package                               Version
------------------------------------- ------------------
absl-py                               1.4.0
accelerate                            1.10.0
aiofiles                              24.1.0
aiohappyeyeballs                      2.6.1
aiohttp                               3.12.15
aiohttp-cors                          0.8.1
aiosignal                             1.4.0
alabaster                             1.0.0
albucore                              0.0.24
albumentations                        2.0.8
ale-py                                0.11.2
altair                                5.5.0
annotated-types                       0.7.0
antlr4-python3-runtime                4.9.3
anyio                                 4.10.0
anywidget                             0.9.18
argon2-cffi                           25.1.0
argon2-cffi-bindings                  25.1.0
array_record                          0.7.2
arviz                                 0.22.0
astrop